In [38]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.animation as animation
from IPython.display import display, HTML
from random import randrange

class Cell:
    def __init__(self, name, direction, row, col):
        self.type = cellTypeDict[name]
        self.name = name
        self.direction = direction
        self.row = row
        self.col = col
    def newStage(self):
        if self.row == 1 and self.col == 1:
            return Cell("head", "r", self.row, self.col)
        if self.name == "end" or self.name == "start" or self.name == "air":
            return self
        return Cell("defwall", "n", self.row, self.col)
        
class CellType:
    def __init__(self, cellID, name, color, fchange, hasDir):
        self.id = cellID
        self.name = name
        self.color = color
        self.fchange = fchange
        self.hasDir = hasDir

lastGrid = []
cellTypeDict = {}
wallList = []
chosenWall = None
isDiceRound = False
gridSize = 0
pathFound = False

def opposite(direction):
    if direction == "u":
        return "d"
    if direction == "d":
        return "u"
    if direction == "l":
        return "r"
    if direction == "r":
        return "l"
    return "n"

def left(direction):
    priority = ["r", "d", "l", "u"]
    i = (priority.index(direction) - 1) % 4
    return priority[i]

def right(direction):
    priority = ["r", "d", "l", "u"]
    i = (priority.index(direction) + 1) % 4
    return priority[i]

def getNeighbor(direction, cell):
    global lastGrid
    row = cell.row
    col = cell.col
    
    if direction == "u":
        return lastGrid[row - 1][col]
    if direction == "d":
        return lastGrid[row + 1][col]
    if direction == "l":
        return lastGrid[row][col + 1]
    if direction == "r":
        return lastGrid[row][col - 1]
    return "none"

def samefunc(cell):
    return cell

def endfunc(cell):
    global pathFound
    if getNeighbor("u", cell).name == "path":
        pathFound = True
    return cell

def tempwallfunc(cell):
    global wallList
    
    un = getNeighbor("u", cell)
    rn = getNeighbor("r", cell)
    dn = getNeighbor("d", cell)
    ln = getNeighbor("l", cell)
    
    direction = "n"
    relevantNeighbor = False
    
    for neighbor in [un, rn, dn, ln]:
        if neighbor.name == "air":
            relevantNeighbor = True 
            if neighbor == un:
                direction = "u"
            elif neighbor == rn:
                direction = "r"
            elif neighbor == dn:
                direction = "d"
            else:
                direction = "l"
    
    if relevantNeighbor:
        wallList.append(cell)
        return Cell("visitedwall", direction, cell.row, cell.col)
    
    return cell

def vwfunc(cell):
    global gridSize, chosenWall
    
    def checkVisited(c):
        if c.name == "air":
            return True
        return False
    
    un = getNeighbor("u", cell)
    rn = getNeighbor("r", cell)
    dn = getNeighbor("d", cell)
    ln = getNeighbor("l", cell)
    
    if cell.row == chosenWall.row and cell.col == chosenWall.col:
        visitedNeighbors = list(filter(checkVisited, [un, rn, dn, ln]))
        wallList.remove(chosenWall)
        if len(visitedNeighbors) > 1:
            return Cell("defwall", "n", cell.row, cell.col)
        else:
            return Cell("air", "n", cell.row, cell.col)
          
    return cell

def airfunc(cell):
    
    un = getNeighbor("u", cell)
    rn = getNeighbor("r", cell)
    dn = getNeighbor("d", cell)
    ln = getNeighbor("l", cell)
    
    for neighbor in [un, rn, dn, ln]:
        if neighbor.name == "feet":
            if neighbor == un:
                if un.direction == "d":
                    return Cell("head", neighbor.direction, cell.row, cell.col)
            elif neighbor == rn:
                if rn.direction == "l":
                    return Cell("head", neighbor.direction, cell.row, cell.col)
            elif neighbor == dn:
                if dn.direction == "u":
                    return Cell("head", neighbor.direction, cell.row, cell.col)
            else:
                if ln.direction == "r":
                    return Cell("head", neighbor.direction, cell.row, cell.col)
                
    return cell

def headfunc(cell):
    priority = ["r", "d", "l", "u"]
    nextCell = getNeighbor(cell.direction, cell)
    leftCell = getNeighbor(left(cell.direction), cell)
    
    def right(direction):
        i = (priority.index(direction) + 1) % 4
        return priority[i]
    
    def isWalkable(c):
        return c.name == "air" or c.name == "path" or c.name == "end"
        
    if isWalkable(leftCell):
        return Cell("feet", left(cell.direction), cell.row, cell.col)
    else:
        if isWalkable(nextCell):
            return Cell("feet", cell.direction, cell.row, cell.col)
    return Cell("head", right(cell.direction), cell.row, cell.col)
    
def feetfunc(cell):
    nextCell = getNeighbor(cell.direction, cell)
    if nextCell.name == "air" or nextCell.name == "end":
        return Cell("path", "n", cell.row, cell.col)
    return Cell("air", "n", cell.row, cell.col)

def render(gridList):
    fig, ax = plt.subplots()
    ax.tick_params(axis='x', which='both', bottom=False, top=False, labelbottom=False)
    ax.tick_params(axis='y', which='both', right=False, left=False, labelleft=False)
    frames = []
    colors = [ct.color for ct in cellTypeDict.values()]
    bounds = [x for x in range(0, len(colors) + 1)]
    cmap = mcolors.ListedColormap(colors)
    norm = mcolors.BoundaryNorm(bounds, cmap.N)
    for grid in gridList:
        idGrid = [[cell.type.id for cell in x] for x in grid]
        img = ax.imshow(idGrid, cmap=cmap, animated=True, norm=norm)
        frames.append([img])

    anim = animation.ArtistAnimation(fig, frames, blit=True, interval=10)
    plt.close()
    display(HTML(anim.to_html5_video()))
    
def initializeDict():
    global cellTypeDict
    cellTypeDict = {
                "air": CellType(0, "air", "white", airfunc, False),
                "tempwall": CellType(1, "tempwall", "black",  tempwallfunc, False),
                "defwall": CellType(2, "defwall", "black", samefunc, False),
                "start": CellType(3, "start", "lime", samefunc, False),
                "end": CellType(4, "end", "red", endfunc, False),
                "visitedwall": CellType(5, "visitedwall", "dimgrey", vwfunc, True),
                "path": CellType(6, "path", "yellow", airfunc, False),
                "head": CellType(7, "head", "aqua", headfunc, True),
                "feet": CellType(8, "feet", "blue", feetfunc, True)
               }

def initializeGrid():
    grid = [[Cell("tempwall", "n", j, i) for i in range(0, gridSize)] for j in range(0, gridSize)]
    grid[0] = [Cell("defwall", "n", 0, i) for i in range(0, gridSize)]
    grid[gridSize - 1] = [Cell("defwall", "n", gridSize - 1, i) for i in range(0, gridSize)]
    grid[0][1] = Cell("start", "n", 0, 1)
    grid[1][1] = Cell("air", "n", 1, 1)
    for row in range(0, gridSize):
        grid[row][0] = Cell("defwall", "n", row, 0)
        grid[row][gridSize - 1] = Cell("defwall", "n", row, gridSize - 1)
    return grid

def createMaze():
    global lastGrid, wallList, chosenWall, isDiceRound
    grids = []
    first = True
    while (len(wallList) != 0) or first == True:
        if len(wallList) != 0 and isDiceRound:
            chosenWall = wallList[randrange(len(wallList))]
        nextGrid = [[cell.type.fchange(cell) for cell in row] for row in lastGrid]
        grids.append(nextGrid)
        first = False
        lastGrid = nextGrid
        isDiceRound = not isDiceRound
    print("Geração do labirinto feita em", len(grids), "frames.")
    return grids

def findPath():
    global lastGrid, pathFound
    grids = []
    while not pathFound:
        nextGrid = [[cell.type.fchange(cell) for cell in row] for row in lastGrid]
        grids.append(nextGrid)
        lastGrid = nextGrid
    print("Caminho encontrado em", len(grids), "frames.")
    return grids
    
def placeExit():
    global lastGrid
    newGrid = lastGrid
    for c in reversed(lastGrid[gridSize-1]):
        n = getNeighbor("u", c)
        if n.name == "air":
            newGrid[c.row][c.col] = Cell("end", "n", c.row, c.col)
            return newGrid
    for row in reversed(range(0, gridSize)):
        c = grid[row][gridSize - 1]
        n = getNeighbor("u", c)
        if n.name == "air":
            newGrid[c.row][c.col] = Cell("end", "n", c.row, c.col)
            return newGrid
    exit(1)
    
def changeStage():
    global lastGrid
    newGrid = [[c.newStage() for c in row] for row in lastGrid]
    return newGrid
    
def maze(size):
    global lastGrid, gridSize
    gridSize = size
    gridList = []
    
    initializeDict()
    
    lastGrid = initializeGrid() 
    firstGrid = lastGrid
    gridList.append(lastGrid)
    gridList = gridList + createMaze()
    lastGrid = placeExit()
    gridList.append(lastGrid)
    lastGrid = changeStage()
    gridList.append(lastGrid)
    gridList = gridList + findPath()
    render(gridList)


In [39]:
maze(40)

Geração do labirinto feita em 2848 frames.
Caminho encontrado em 2275 frames.
